# Extracción de Tweets

En esta notebook se van a revisar los pasos para lograr un buen minado de tweets. Para ello se tocan los siguientes puntos: 
- Qué es la API de Twitter y cómo utilizarla
- Operadores para definir queries
- Ejemplos de minado

**TODO: Porque es util minar de Twitter?**

Twitter provee una API (interfaz de programación de aplicaciones) para acceder a sus datos. Las API son mecanismos que permiten a dos componentes de software comunicarse entre sí mediante un conjunto de definiciones y protocolos. Por ejemplo, el sistema de software del instituto de meteorología contiene datos meteorológicos diarios. La aplicación meteorológica de su teléfono “habla” con este sistema a través de las API y le muestra las actualizaciones meteorológicas diarias en su teléfono. [Aquí una explicación de AWS](https://aws.amazon.com/es/what-is/api/).

## Requerimientos

<h3 id='credentials_api'>Credenciales para la API de Twitter</h3>

Para obtener tweets con la [API Oficial de Twitter](https://developer.twitter.com/en/docs/twitter-api) es necesario registrarse en el [portal para desarrolladores de Twitter](https://developer.twitter.com/en/docs/developer-portal/overview) para así dar de alta un proyecto y que se te asignen las [credenciales necesarias](https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api).

Hay que aclarar que **según el tipo de credenciales y el método de autenticación serán diferente la [cantidad de tweets que podremos obtener en cierto tiempo](https://developer.twitter.com/en/docs/twitter-api/rate-limits#v2-limits), qué [operadores/filtros podemos usar](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list) y cuantos días hacia atrás podremos minar** y la extensión de los queries. 

Por ejemplo, los límites mensuales de obtención de tweets, dependiendo de la credencial empleada, son los siguientes:
- *Essential*: 500k tweets en un mes y 512 caracteres por query.
- *Elevated*: 2M tweets en un mes y 512 caracteres por query.
- *Academic*: 10M tweets en un mes y 1024 caracteres por query.

Cualquier otra diferencia entre nivel de credenciales, se puede [ver aquí](https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-level)

Notas:
- Debido a que la web app del Laboratorio de Migración solo necesita permisos de lectura para contenido público, nos interesa obtener la credencial de tipo Bearer Token, ya que es el método de autenticación que más tweets nos permite obtener en menos tiempo.
- Se usará la versión 2 de la [API de Twitter](https://developer.twitter.com/en/docs/twitter-api).
- El ejercicio aquí propuesto puede realizarse con las credenciales de nivel [*Elevated*](https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-level), pero en caso de estar trabajando en un proyecto con fines académicos o de estudio, se puede [aplicar de manera gratuita a credenciales Académicas](https://developer.twitter.com/en/products/twitter-api/academic-research) y así trabajar con límites más amplios y operadores avanzados.

### Dependencias de Python
Las dependencias principales son `twarc`, `pandas` y `numpy`. Las últimas dos son para manipular y trabajar con los datos, mientras que [`twarc` es el paquete](https://developer.twitter.com/en/docs/twitter-api/rate-limits) que nos facilitará el proceso de minado, ya que se encarga de obtener automáticamente todos los atributos de los tweets así como de manejar los tiempos de espera cuando se llega a los [límites de minado de la API](https://developer.twitter.com/en/docs/twitter-api/rate-limits).

In [1]:
# Indispensables
!pip install twarc==2.9.2
!pip install pandas==1.4.1
!pip install numpy==1.22.3

# Para mejor interacción gráfica
!tqdm==4.62.2
!pendulum==2.1.2

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consider upgrading via the '/home/noecampos/.pyenv/versions/BID/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consider upgrading via the '/home/noecampos/.pyenv/versions/BID/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consider upgrading via the '/home/noecampos/.pyenv/versions/BID/bin/python -m pip install --upgrade pip' command.


## Imports & Credenciales

In [2]:
import os
import json
import pendulum
import pandas as pd


from tqdm import tqdm

from twarc.client2 import Twarc2
from twarc.expansions import TWEET_FIELDS
from twarc.expansions import ensure_flattened

Ingresa tu propio Bearer Token o coméntalo, y descomenta el resto de atributos para con tus propias API Keys y Access Tokens.

In [3]:
CREDENTIALS_TWITTER_API = {
    'bearer_token': "Enter your own Bearer token",

    # 'api_key': "Enter your own API Key",
    # 'api_secret_key': "Enter your own API Secret Key",
    # 'access_token': "Enter your own access_token",
    # 'access_token_secret': "Enter your own access_token_secret"
}

IS_ACADEMIC = False # Cambiar a True, si las credenciales son Academicas

## Minado de Tweets

La API de Twitter permite obtener diferentes piezas de información a partir de los usuarios y los tweets que publican, según el tipo de operadores y queries utilizados. Para fines de esta notebook, nos centraremos en obtener tweets públicos en un intervalo de tiempo definido, a partir del contenido de palabras claves.

Las palabras claves del tema que se desea minar se utilizan para construir queries que hacen la búsqueda más certera.

<h3 id='endpoints_limits'>Endpoints & Límites</h3>

Para [buscar tweets](https://developer.twitter.com/en/docs/twitter-api/tweets/search/introduction) hay dos endpoints, [Recent Search](https://developer.twitter.com/en/docs/twitter-api/tweets/search/quick-start/recent-search) y  [Full Archive](https://developer.twitter.com/en/docs/twitter-api/tweets/search/quick-start/full-archive-search).
- **Recent Search**: Nos permite realizar 450 requests (pedidos) a la API en una ventana de 15 minutos, obteniendo máximo de 100 tweets por request. Sin embargo, solo se pueden obtener tweets publicados en los últimos 7 días.
- **Full Archive**: Podremos obtener tweets publicados desde el inicio de la red social, pero solo se pueden realizar 300 requests en una ventana de 15 minutos, obteniendo hasta 500 tweets por cada request. Hay que aplicar a las [credenciales académicas](https://developer.twitter.com/en/products/twitter-api/academic-research) para poder utilizar este endpoint.

**Nota**: No hay que preocuparse del código de error que aparece una vez que el minado alcanza estos límites, pues el paquete twarc se encarga de pausar la obtención de tweets una vez que se llega al límite de 450 (o 300) requests en la ventana de 15 minutos; una vez que pasa un tiempo necesario, twarc reanuda el proceso. Si se llega al límite de tweets en un mes, la función para.

### Operadores

En esta sección vamos a ver de manera general qué son y cómo funcionan los operadores, pero la lista y descripción completa de los mismos se puede [encontrar aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query).

Existen dos tipos de operadores:
- **Standalone**: Se pueden usar solos o en conjunción de otros. Por ejemplo, buscar tweets con un hashtag en específico: `#migrantes`.
- **Conjunction-required**: Es necesario que estén junto a mínimo un operador *standalone*. Por ejemplo, buscar tweets con un hashtag en específico, pero que incluyan imágenes y sean retweets: `#migrantes has:media is:retweet`

Además, como se mencionó en la sección de "Credenciales para la API de Twitter", existen operadores *core*, que son accesibles con cualquier nivel de acceso, así como los operadores *advanced*, que solo se pueden utilizar con un acceso académico.

#### Operadores Lógicos

- **AND**: Obtiene tweets que cumplan con los dos operadores, se logra dejando un espacio en blanco entre ellos. 
    - Ejemplo, obtener tweets que contienen la palabra *políticos* y el hashtag *#corruptos*: `politicos #corruptos`.
- **OR**: Obtiene tweets que cumplan con alguno de los dos operadores. Hay que añadir el string " OR " entre los operadores. 
    - Ejemplo, tweets que contengan la palabra *migrantes* o *inmigrantes*: `migrantes OR inmigrantes`.
- **NOT**: Obtiene tweets que no contengan el operador o la keyword negada. Se logra añadiendo un guion medio "-" antes del operador o keyword. 
    - Ejemplo, obtener tweets con la palabra *políticos* pero sin la palabra *corruptos*: `politicos -corruptos`
    - Ejemplo, obtener tweets con la palabra *migrantes, pero que no sean retweets*: `migrantes -is:retweets`
- **Grouping**: Sirve para agrupar operadores lógicos, y hay que encerrar los operadores entre parentesis. Un grupo no puede ser negado.
    - Ejemplo, obtener tweets con la palabra *migrantes* y alguna de las palabras *llegan* o *salen*: `migrantes AND (llegan OR salen)`

**Nota**: A menos de que haya paréntesis para especificar el orden de operadores, primero se resuelven aquellos que son *AND* y luego los *OR*. [Más aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#boolean).

#### Más Operadores

- **keyword**: Hace match con tweets que contengan un string en específico. No es sensible a caracteres en mayúsculas o minúsculas; acentos o caracteres especiales como ñ. Ejemplo: `migrantes OR inmigrantes`.
- **"exact phrase"**: Parecido al anterior, permite considerar espacios y múltiples tokens. Tiene que estar entre comillas dobles. Ejemplo: `"ola migrante"`.
- **#**: Hace match a tweets que tengan el hashtag incluido. Ejemplo: `#migraresunderecho`.
- **@**: Hace match a tweets que mencionen a los usuarios incluidos. Ejemplo: `@IADB`.
- **place_country**: Obtiene tweets que sean geo-localizable a cierto país. Hay que pasarle el código [ISO del país](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2). Ejemplo: `place_country:MX`.
- **lang**: Obtiene tweets que estén escrito en el lenguaje indicado y tiene que estar junto con un standalone operator. Ejemplo: `migrantes lang:es`.
- **is:retweet**: Obtiene solo tweets que sean retweets y tiene que estar junto con un standalone operator. Ejemplo: `migrantes is:retweet`.

La lista completa de operadores se puede ver [aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list).

### Ejercicio de Minado

La siguiente función recibe una lista de queries y un intervalo de tiempo para guardar los tweets en un archivo JSONL.

**Nota**: Un archivo de extension .jsonl guarda un objeto json por cada línea separados por solo el salto de línea, no comas u otro separador.

In [5]:
def get_tweets(credentials_api,
               queries_list,
               output_file,
               since_date, until_date,
               is_academic=False):
    """Function in charge of scrape tweets from the 
    official Twitter API, using the library named Twarc.

    Args:
        credentials_api (dict): Dictionary with the Twitter API credentials.
        queries_list (list[str]): List of queries to scrape.
        output_file (str): Path to the file in which to store the results.
        since_date (datetime): Start of the time span to scrape.
        until_date (datetime): End of the time span to scrape.
        is_academic (bool, optional): If the credentials has Research 
                                      Academic access level.
    """

    # Instiate the Twarc Client
    twarc_client = Twarc2(**credentials_api)

    # Make some tweaks for using the research credentials
    max_size = 100
    tweet_fields = TWEET_FIELDS.copy()
    search_func = twarc_client.search_recent
    if(is_academic):
        search_func = twarc_client.search_all
        max_size = 500

        # Remove the context_annotations attr to
        # scrape 500 tweets per request
        tweet_fields.remove('context_annotations')

    tweet_fields = ','.join(tweet_fields)

    with open(output_file, 'a') as pages_file:
        for query in tqdm(queries_list):

            search_results = search_func(query=query,
                                         start_time=since_date,
                                         end_time=until_date,
                                         tweet_fields=tweet_fields,
                                         max_results=max_size)

            # Write all the obtained tweets
            for page in search_results:

                # Write one by one the tweets
                for tweet in ensure_flattened(page):
                    json.dump(tweet, pages_file, ensure_ascii=False)
                    pages_file.write('\n')

Se declara la lista de queries, el intervalo de fecha (para fines de este notebook, es 6 h hacia atrás de la hora actual) y si las credenciales que se están utilizando tienen un nivel de acceso académico. Esto último es útil para pedir que cada request tenga 500 tweets.


Para este ejemplo, se minarán tweets que:
- Contengan "migrantes" y "migración", o alguna de sus variantes
- Haya sido publicado en las últimas 3 horas

Y se va a guardar en un archivo llamado `1_tweets_test.jsonl`.

Algo a tomar en cuenta, es que la API de Twitter no acepta expresiones regulares o wildcards, es decir, no podemos pasarle un string como `(in|e)?migrantes` esperando que obtenga tweets con las palabras "inmigrantes", "emigrantes" o "migrantes". Hay que pasarle todas las variaciones de la keyword que esperamos obtengan información útil.

In [6]:
lst_queries = ['migrante', 'inmigrante', 'emigrante', 
               'migrantes', 'inmigrantes', 'emigrantes',
               'migración', 'inmigracion', 'emigracion']

# De las ultimas 24 hrs
date_end = pendulum.today()
date_start = date_end.subtract(hours=3)

# # O un rango de fechas definido
# date_start = pendulum.datetime(year=2022, month=11, day=14)
# date_end = pendulum.datetime(year=2022, month=11, day=13)

file_tweets = os.path.abspath("./files/1_tweets_test.jsonl")

In [7]:
# Borrar el archivo si ya existe
if(os.path.exists(file_tweets)):
    os.remove(file_tweets)

In [8]:
get_tweets(credentials_api = CREDENTIALS_TWITTER_API,
           queries_list = lst_queries,
           output_file = file_tweets,
           since_date = date_start, 
           until_date = date_end,
           is_academic= IS_ACADEMIC)

100%|██████████| 9/9 [00:43<00:00,  4.81s/it]


Con el siguiente comando (de bash) se puede revisar cuantos tweets obtuvimos

In [9]:
!wc -l ./files/1_tweets_test.jsonl

3388 ./files/1_tweets_test.jsonl


Es posible que existan tweets que contengan dos o más de las keywords, por lo que al hacer búsquedas diferentes, obtendremos N veces el mismo tweet. Por ejemplo, aquellos tweets que contengan las palabras "migrante" y "migrantes" los obtendremos dos veces. 

Esto se puede optimizar usando el operador lógicos `OR`, así podremos hacer request que aprovechen la longitud máxima de los queries, reducir el número de request y tiempo de minado, ya que estaremos obteniendo más tweets por request. Los límites de la longitud en caracteres de los queries se vieron en la sección [Credenciales para la API de Twitter](#credentials_api) y los request/tiempos en [Endpoints & Límites](#endpoints_limits).

In [10]:
# Juntar las keywords en un solo string
new_query = ' OR '.join(lst_queries)
new_query

'migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes OR migración OR inmigracion OR emigracion'

En lugar de 9 queries ahora tenemos solo 1 compuesto por 80 caracteres (de los 512 o 1024 posibles)

In [11]:
# Hay que poner el nuevo query dentro de una lista dado que es lo que espera la funcion
lst_queries_2 = [new_query]

# Nuevo archivo
file_tweets = os.path.abspath("./files/2_tweets_test.jsonl")

if(os.path.exists(file_tweets)):
    os.remove(file_tweets)

In [12]:
get_tweets(credentials_api = CREDENTIALS_TWITTER_API,
           queries_list = lst_queries_2,
           output_file = file_tweets,
           since_date = date_start, 
           until_date = date_end,
           is_academic= IS_ACADEMIC)

100%|██████████| 1/1 [00:35<00:00, 35.23s/it]


In [13]:
!wc -l ./files/2_tweets_test.jsonl

3243 ./files/2_tweets_test.jsonl


Esta optimización con operadores lógicos se puede extender a cuando necesitamos tweets que contengan dos o más posibles palabras. Por ejemplo, obtener los tweets que contengan alguna de las keywords: "migrantes", "inmigrantes" y la keyword "bienvenidos".

El query sería `bienvenidos (migrantes OR inmigrantes)`, recordando que:
- Los paréntesis agrupan el OR 
- El espacio entre la primera keyword y los parentesis es un `AND`

Si se quiere añadir otra keyword opción a "bienvenidos", sería `(hermanos OR bienvenidos) (migrantes OR inmigrantes)`. Las combinaciones que forma son: `hermanos AND migrantes`, "hermanos inmigrantes", "bienvenidos migrantes" y "bienvenidos inmigrantes".

### Ejemplo final de queries

Como un ejemplo más completo, vamos a obtener tweets que:
- Contengan alguna de las siguientes palabras: "migrantes", "inmigrantes" o "emigrantes"
- O que tengan alguno de estos hashtags: #migraresunderecho,  #todossomomigrantes o #heramanomigrante
- Que estén en español
- Que hayan sido publicados por alguien en México o en Argentina
- Y fuesen publicados en los últimos 5 días


Nota: Dado que se están añadiendo filtros de lenguajes y sobre todo de país, se espera obtener un volumen menor de tweets a que si solo fueran los keywords y hashtags.

In [14]:
keywords_part = "(migrantes OR inmigrantes OR emigrantes)"
hashtags_part = "(#migraresunderecho OR #todossomomigrantes OR #hermanomigrante)"
language_part = "lang:es"
country_part = "(place_country:MX OR place_country:AR)"

In [15]:
final_query = f"({keywords_part} OR {hashtags_part}) {language_part} {country_part}"
final_query

'((migrantes OR inmigrantes OR emigrantes) OR (#migraresunderecho OR #todossomomigrantes OR #hermanomigrante)) lang:es (place_country:MX OR place_country:AR)'

In [16]:
# Hay que poner el nuevo query dentro de una lista dado que es lo que espera la funcion
lst_queries_3 = [final_query]

# De las ultimas 5 dias
date_end = pendulum.today()
date_start = date_end.subtract(days=5)

# Nuevo archivo
file_tweets = os.path.abspath("./files/3_tweets_test.jsonl")

if(os.path.exists(file_tweets)):
    os.remove(file_tweets)

In [17]:
get_tweets(credentials_api = CREDENTIALS_TWITTER_API,
           queries_list = lst_queries_3,
           output_file = file_tweets,
           since_date = date_start, 
           until_date = date_end,
           is_academic= IS_ACADEMIC)

100%|██████████| 1/1 [00:03<00:00,  3.06s/it]


In [18]:
!wc -l ./files/3_tweets_test.jsonl

72 ./files/3_tweets_test.jsonl


### Objeto Tweet
Antes de terminar esta notebook, hay que revisar que campos contiene el [objeto tweet](https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet).

Los campos más importantes son:
- **id**: Identificador unico de cada tweet
- **conversation_id**: Identificador unico del tweet que inicio la conversación
- **created_at**: Fecha y hora en la que se publicó el tweet
- **text**: Texto publicado en el tweet
- **possibly_sensitive**: Si el texto incluye links a contenido posiblemente sensible
- **lang**: Idioma en el que está escrito el tweet
- **source**: Sí fue publicado desde un iPhone, Android, un buscador, etc.
- **geo**: Objeto que contiene el lugar, país y coordenadas que se asignaron al tweet
- **author_id**: Identificador único del autor
- **author**: Objeto que incluye atributos del autor
- **public_metrics**: Objeto que tiene el conteo de retweets, replies, likes y quotes



```json
{
    "id": "1589089368777850880",
    "conversation_id": "1589089368777850880",
    "created_at": "2022-11-06T02:56:44.000Z",
    "text": "Primer vuelo humanitario para repatriar a venezolanos que estaban en #México \n\n140 personas regresaron voluntariamente en un vuelo coordinado por @INAMI_mx \n\nSegún las autoridades mexicanas continuarán apoyando el regreso de migrantes a su país. https://t.co/JuUjRB3oQk",
    "possibly_sensitive": false,
    "lang": "es",
    "source": "Twitter for iPhone",

    "geo": {
        "place_id": "2a376531dff3d76a",
        "full_name": "Tlalpan, Distrito Federal",
        "name": "Tlalpan",
        "place_type": "city",
        "id": "2a376531dff3d76a",
        "country": "México",
        "geo": {
            "type": "Feature",
            "bbox": [-99.315748, 19.087511, -99.1009804, 19.311459],
            "properties": {}
        },
        "country_code": "MX"
    },

    "author_id": "114573824",
    "author": {
        "verified": false,
        "protected": false,
        "profile_image_url": "https://pbs.twimg.com/profile_images/1569749213822570500/Xf8yCPws_normal.jpg",
        "created_at": "2010-02-15T21:50:44.000Z",
        "pinned_tweet_id": "1588664226729832448",
        "url": "https://t.co/E1sEHNlBkT",
        "description": "Periodista mexicano 🗞 @aztecanoticias",
        "public_metrics": {
            "followers_count": 11893,
            "following_count": 732,
            "tweet_count": 24530,
            "listed_count": 25
        },
        "entities": {
            "url": {
                "urls": [{
                    "start": 0,
                    "end": 23,
                    "url": "https://t.co/E1sEHNlBkT",
                    "expanded_url": "http://www.facebook.com/OtonielMartínez",
                    "display_url": "facebook.com/OtonielMartínez"
                }]
            },
            "description": {
                "mentions": [{
                    "start": 22,
                    "end": 37,
                    "username": "aztecanoticias"
                }]
            }
        },
        "name": "OTONIEL MARTÍNEZ",
        "username": "_otomartinez",
        "location": "MÉXICO",
        "id": "114573824"
    },

    "public_metrics": {
        "retweet_count": 4,
        "reply_count": 1,
        "like_count": 19,
        "quote_count": 0
    },
    "reply_settings": "everyone",
    "attachments": {
        "media_keys": ["3_1589089086119333889", "3_1589089086245146626"],
        "media": [{
            "type": "photo",
            "media_key": "3_1589089086119333889",
            "width": 774,
            "url": "https://pbs.twimg.com/media/Fg2UGr6X0AEKzBx.jpg",
            "height": 1024
        }, {
            "type": "photo",
            "media_key": "3_1589089086245146626",
            "width": 731,
            "url": "https://pbs.twimg.com/media/Fg2UGsYXkAIC8S4.jpg",
            "height": 1024
        }]
    },

    "entities": {
        "annotations": [{
            "start": 70,
            "end": 75,
            "probability": 0.9717,
            "type": "Organization",
            "normalized_text": "México"
        }],
        "urls": [{
            "start": 246,
            "end": 269,
            "url": "https://t.co/JuUjRB3oQk",
            "expanded_url": "https://twitter.com/_otomartinez/status/1589089368777850880/photo/1",
            "display_url": "pic.twitter.com/JuUjRB3oQk",
            "media_key": "3_1589089086119333889"
        }, {
            "start": 246,
            "end": 269,
            "url": "https://t.co/JuUjRB3oQk",
            "expanded_url": "https://twitter.com/_otomartinez/status/1589089368777850880/photo/1",
            "display_url": "pic.twitter.com/JuUjRB3oQk",
            "media_key": "3_1589089086245146626"
        }],
        "hashtags": [{
            "start": 69,
            "end": 76,
            "tag": "México"
        }],
        "mentions": [{
            "start": 146,
            "end": 155,
            "username": "INAMI_mx",
            "id": "1300283125",
            "verified": true,
            "protected": false,
            "profile_image_url": "https://pbs.twimg.com/profile_images/1542852123821424640/exw6RvmZ_normal.jpg",
            "created_at": "2013-03-25T17:02:40.000Z",
            "pinned_tweet_id": "1589088702302945280",
            "url": "https://t.co/aMyjqh7MV3",
            "description": "Instituto Nacional de Migración",
            "public_metrics": {
                "followers_count": 52464,
                "following_count": 656,
                "tweet_count": 27656,
                "listed_count": 334
            },
            "entities": {
                "url": {
                    "urls": [{
                        "start": 0,
                        "end": 23,
                        "url": "https://t.co/aMyjqh7MV3",
                        "expanded_url": "https://www.gob.mx/inm",
                        "display_url": "gob.mx/inm"
                    }]
                }
            },
            "name": "INM",
            "location": "México"
        }]
    }
}
```

## Como manipular el JSONL a un DataFrame